New predictor = Normalized avoidable cost + Normalized Health Records (gagne sum) as the new predictor

New cost = Normalized total cost (higher expense from allocating the enrollment)

Experiment Setup

1. Group-wise prediction fairness + decision
2. accuracy parity + Alpha-Fairness group version
3. Individual prediction fairness + decision

- 2-Stage
- Closed-Form
- Fold-Opt (PGD via closed form)
- Finite Difference
- cvxpylayer* 


In [1]:
import sys
import warnings
import copy
import json
from datetime import datetime

import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler

import cvxpy as cp

from pyepo.model.opt import optModel
sys.path.insert(0, 'E:\\User\\Stevens\\MyRepo\\FDFL\\helper')
sys.path.insert(0, 'E:\\User\\Stevens\\MyRepo\\fold-opt-package\\fold_opt')

from myutil import *
from features import get_all_features

# Suppress warnings
warnings.filterwarnings("ignore")

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
df = pd.read_csv('E:\\User\\Stevens\\MyRepo\\FDFL\\data\\data.csv')
# fix random seed for reproducibility

# report statistics on this dataset
# df = df.sample(n=5000, random_state=1)

columns_to_keep = [
    'risk_score_t', 'program_enrolled_t', 'cost_t', 'cost_avoidable_t', 'race', 'dem_female', 'gagne_sum_tm1', 'gagne_sum_t', 
    'risk_score_percentile', 'screening_eligible', 'avoidable_cost_mapped', 'propensity_score', 'g_binary', 
    'g_continuous', 'utility_binary', 'utility_continuous'
]
# # for race 0 is white, 1 is black
# df_stat = df[columns_to_keep]
# df_feature = df[[col for col in df.columns if col not in columns_to_keep]]

# # Replace all values less than 0.1 with 0.1
# #df['risk_score_t'] = df['risk_score_t'].apply(lambda x: 0.1 if x < 0.1 else x)
# df['g_continuous'] = df['g_continuous'].apply(lambda x: 0.1 if x < 0.1 else x)

# risk = df['risk_score_t'].values
# risk = risk + 0.001 if 0 in risk else risk

# feats = df[get_all_features(df)].values
# gainF = df['g_continuous'].values
# decision = df['propensity_score'].values
# cost = np.random.normal(1, 0.5, len(risk)).clip(0.1, 2)
# race = df['race'].values

# # transform the features
# scaler = StandardScaler()
# feats = scaler.fit_transform(feats)



Auto-Sklearn cannot be imported.
cuda


In [2]:
def printStats(array):
    print(f"Min: {np.min(array)}")
    print(f"Max: {np.max(array)}")
    print(f"Mean: {np.mean(array)}")
    print(f"Std: {np.std(array)}")
    print(f"Median: {np.median(array)}")
    print(f"Quantiles: {np.quantile(array, [0.25, 0.5, 0.75])}")
    print(f"Sum: {np.sum(array)}")
    print(f"Length: {len(array)}")

Assume that patients with higher need (higher cost and chronic conditions) have more potential to benefit

$C_{norm} = log(1+C)$.

then normalize $C$ to 0-1.

For active chronic conditions $H$, normalize to 0-1.

The potential benefit $B = (w_c C + w_h H)$, where $w$ are weights.

In [3]:
w_c = 0.5
w_h = 0.5

# Step 1: Log-transform the avoidable cost to reduce skew
df['cost_log'] = np.log1p(df['cost_avoidable_t'])  # log(1 + cost)

# Step 2: Normalize the transformed cost
cost_min = df['cost_log'].min()
cost_max = df['cost_log'].max()
df['cost_norm'] = (df['cost_log'] - cost_min) / (cost_max - cost_min)

# Step 3: Normalize the active chronic conditions count
max_chronic = 17 
df['gagne_norm'] = df['gagne_sum_t'] / max_chronic

# Step 4: Compute benefit variable (higher benefit = better)
df['benefit'] = (w_c * df['cost_norm'] + w_h * df['gagne_norm'])

# Optional: Check the range and stats of the benefit
printStats(df['benefit'].values)


Min: 0.0
Max: 0.9221911333091197
Mean: 0.11476308021250582
Std: 0.16024239541756805
Median: 0.029411764705882353
Quantiles: [0.         0.02941176 0.21337489]
Sum: 5598.602105086884
Length: 48784


#### We defined a benefit variable to represent the potential outcome one is to get from enrollment `benefit`
Next, we should use the total cost as a proxy to represent the cost of distributing resources (in the constraint) `cost_t_capped`

In [4]:
cost_99 = df['cost_t'].quantile(0.99)
df['cost_t_capped'] = df['cost_t'].clip(upper=cost_99)

# normalize the capped cost to [0, 1]
df['cost_t_capped'] = (df['cost_t_capped'] - df['cost_t_capped'].min()) / (df['cost_t_capped'].max() - df['cost_t_capped'].min())
df['cost_t_capped'] = df['cost_t_capped']


df.to_csv('E:\\User\\Stevens\\MyRepo\\Organized-FDFL\\src\\data\\data.csv', index=False)

In [5]:
# 1. Min–max normalize to [0,1]
b_min, b_max = df['benefit'].min(), df['benefit'].max()
c_min, c_max = df['cost_t_capped'] .min(), df['cost_t_capped'] .max()
b_norm = (df['benefit'] - b_min) / (b_max - b_min)
c_norm = (df['benefit'] - c_min) / (c_max - c_min)

In [ ]:
# C_tot = c_norm.sum()

# # 3. Choose coverage fraction ρ
# rho = 0.5   # e.g. serve 40% of total cost

# # 4. Set budget
# Q = rho * C_tot

# print(f"Normalized range → b∈[0,1], c∈[0,1];  Q = {rho:.2f}·C_tot = {Q:.3f}")

Normalized range → b∈[0,1], c∈[0,1];  Q = 0.50·C_tot = 2799.301


We have 
1. `df['cost_t_capped']` as `c`, normalized 0-1
2. `df['benefit']` as `b`, normalized 0-1
